In [4]:
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import torch.utils.data as Data
import torch.optim as optim
from tqdm import tqdm
from torch.autograd import Variable
import time
import multiprocessing as mp
from os import walk
day = 30
stride = 10
feat_num = 11
conv_num = int(feat_num * (feat_num - 1)/2)

In [9]:
data_time = "2021-01-01_2021-03-01.par"

In [75]:
data = pd.read_parquet(path + "Decay" +"/" + data_time)
data['timestamp'] = pd.to_datetime(data['timestamp'])
data = data[data['timestamp'] <= pd.to_datetime("20210105")]
data.to_parquet(path + "test" +"/" + "Decay.par")
data

,timestamp,ticker,0,1,2,3,4,5,6,7,...,17,18,19,20,21,22,23,24,25,26
0,2021-01-04 09:30:00,1,2185.902545,2089.372909,2065.567636,2228.042727,2111.057455,2103.141818,2142.169273,2051.636000,...,913361.494364,0.112375,0.059224,0.831236,0.480658,0.385069,0.470660,1.084467,0.868782,1.061904
35,2021-01-04 09:30:00,2,4785.078000,4442.552727,4347.245636,4864.633636,4467.176909,4395.038545,4703.625636,4376.712364,...,548741.821091,-0.024753,-0.167056,0.275409,0.795204,0.575093,0.564871,1.193689,0.863313,0.847933
70,2021-01-04 09:30:00,4,115.112545,98.471091,86.813091,116.368000,100.353636,88.367091,111.797273,95.857455,...,27253.327273,-1.876227,-1.153982,-0.957409,6.133796,4.669475,3.247602,8.573011,6.526384,4.539049
105,2021-01-04 09:30:00,5,24.205818,23.567818,23.285636,24.433273,23.806727,23.443455,23.934364,23.319455,...,62101.198909,-0.216373,-0.030745,0.055396,1.150500,0.662275,0.586975,1.384855,0.797180,0.706578
140,2021-01-04 09:30:00,6,209.427636,200.409455,202.645818,211.723091,202.447455,206.820182,207.810182,197.031091,...,81700.248000,0.261849,-0.093322,0.147818,0.932264,0.505765,0.605945,1.457684,0.790811,0.947447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142085,2021-01-04 09:30:00,688160,56.033636,48.011455,42.055818,57.014000,48.924000,43.026182,54.129636,46.389818,...,8589.565455,-2.145462,-0.526427,-1.183189,15.784753,8.117129,4.500498,15.784753,8.117129,4.500498
142120,2021-01-04 09:30:00,3019,39.044182,34.530364,30.134727,39.889636,35.090545,30.596545,38.142909,33.878545,...,18612.216364,-1.088462,-0.658796,-0.492465,19.856029,7.288125,5.816309,19.856029,7.288125,5.816309
142155,2021-01-04 09:30:00,300907,51.971455,44.412909,37.781273,53.323455,45.162727,38.726909,50.666727,43.177455,...,27897.146182,-1.583740,-1.457129,-1.108291,27.038420,14.126827,12.256036,27.038420,14.126827,12.256036
142190,2021-01-04 09:30:00,300909,74.238727,64.288000,54.736545,75.783818,65.848545,55.959455,72.338909,62.526727,...,24875.512182,-1.396007,-0.787635,-1.353676,20.990669,13.577111,10.397216,20.990669,13.577111,10.397216


In [64]:
data = data.set_index(["timestamp","ticker"])
x = np.array(data.drop('target',axis=1))
x = x.reshape(-1,30,11)
target = data['target']
data

KeyError: "['target'] not found in axis"

In [74]:
data = data.set_index(["timestamp","ticker"])
x = np.array(data)
x = x.reshape(x.shape[0],-1,3)
data

0         1         2         3         4  \
timestamp           ticker                                                     
2021-01-04 09:30:00 1       0.050448 -0.028763  0.011052  0.045812 -0.020545   
                    2       0.014881 -0.080655 -0.019072  0.012238 -0.070970   
                    4      -0.188767 -0.119012 -0.107052 -0.201920 -0.115510   
                    5      -0.026369 -0.026971 -0.015718 -0.018593 -0.023150   
                    6       0.006931 -0.055068  0.014623  0.010361 -0.051537   
...                              ...       ...       ...       ...       ...   
                    688160 -0.207207 -0.097209 -0.145623 -0.289157 -0.090599   
                    3019   -0.145543 -0.096913 -0.131587 -0.141340 -0.087108   
                    300907 -0.170968 -0.158534 -0.143704 -0.187244 -0.157081   
                    300909 -0.128059 -0.131714 -0.172359 -0.128417 -0.101105   
                    688219 -0.199336 -0.140316 -0.154955 -0.248805 -0.153495   

                                   5         6         7         8         9  \
timestamp           ticker                                                     
2021-01-04 09:30:00 1       0.025129  0.035096 -0.019501  0.043336  0.036054   
                    2      -0.001732  0.003614 -0.054777 -0.003878  0.000972   
                    4      -0.089741 -0.152880 -0.128382 -0.096541 -0.160501   
                    5      -0.015533 -0.018876 -0.030949 -0.011940 -0.025869   
                    6       0.027145  0.003484 -0.066189  0.010987  0.008688   
...                              ...       ...       ...       ...       ...   
                    688160 -0.161382 -0.199845 -0.127027 -0.152357 -0.288843   
                    3019   -0.113335 -0.126365 -0.087363 -0.099113 -0.166777   
                    300907 -0.161991 -0.163228 -0.173638 -0.135565 -0.176267   
                    300909 -0.155019 -0.115269 -0.141509 -0.161301 -0.131097   
                    688219 -0.142478 -0.202830 -0.162948 -0.139578 -0.261579   

                            ...        17        18         19         20  \
timestamp           ticker  ...                                             
2021-01-04 09:30:00 1       ... -0.126802 -9.634652  -0.743039  -1.234206   
                    2       ...  0.240802  4.709428  -1.288127  -1.956485   
                    4       ... -0.215357 -1.031973  -1.152113  -1.510681   
                    5       ...  0.108007 -0.613900  -1.000000  -5.095869   
                    6       ...  0.185126 -1.171500  -1.530022   0.263500   
...                         ...       ...       ...        ...        ...   
                    688160  ... -0.312674 -1.177026  -0.269697  -0.675929   
                    3019    ... -0.296164 -1.475350  -1.487843  -1.470363   
                    300907  ... -0.411749 -0.548944  -2.534892  12.846479   
                    300909  ... -0.312700 -0.211864  -3.682839   0.941669   
                    688219  ... -0.356844  0.915804  11.784537   0.360063   

                                  21        22        23        24        25  \
timestamp           ticker                                                     
2021-01-04 09:30:00 1       0.060179 -0.230133 -0.126833  0.060243 -0.230242   
                    2       0.177101 -0.360985  0.240895  0.176949 -0.361016   
                    4      -0.775437  0.175471 -0.215357 -0.775435  0.175467   
                    5       1.059912  0.016669  0.107865  1.060082  0.016723   
                    6       0.730633 -0.345320  0.185228  0.730361 -0.345306   
...                              ...       ...       ...       ...       ...   
                    688160 -0.799648 -0.122794 -0.312668 -0.799648 -0.122794   
                    3019   -0.537271 -0.489945 -0.296171 -0.537271 -0.489945   
                    300907 -0.592859 -0.024531 -0.411750 -0.592859 -0.024531   
                    300909 -0.544904  0.285569 -0.312698 -0.544904  0.2

In [67]:
x.shape

(4065, 36, 3)

In [36]:
print(data.shape)
print(x.shape)
print(target.shape)

(3745, 331)
(3745, 30, 11)
(3745,)


In [43]:
for i in range(len(data.index)):
    print(data.index[i])
    print("data")
    display(pd.DataFrame(x[i]))
    print("target",target[i])
    print("********************************************************")

(Timestamp('2021-01-04 09:30:00'), 1)
data


,0,1,2,3,4,5,6,7,8,9,10
0,18.85,18.83,18.99,18.52,18.86,18.7747,673919.22,0.0531,0.3473,0.7835,0.01
1,18.86,18.85,19.62,18.80,19.62,19.3187,1165858.26,4.0297,0.6008,1.3555,0.76
2,19.62,19.62,19.68,19.17,19.36,19.3576,678543.23,-1.3252,0.3497,0.7889,-0.26
3,19.36,19.48,19.70,19.05,19.06,19.3337,552585.01,-1.5496,0.2848,0.6425,-0.30
4,19.06,19.05,19.61,19.03,19.50,19.3865,639657.89,2.3085,0.3296,0.7437,0.44
5,19.50,20.00,20.00,19.38,19.70,19.6270,753773.74,1.0256,0.3884,0.8764,0.20
6,19.70,19.90,20.88,19.59,19.74,20.3212,1581441.28,0.2030,0.8149,1.8386,0.04
7,19.74,19.70,20.51,19.40,20.05,19.9096,1263719.75,1.5704,0.6512,1.4693,0.31
8,20.05,19.93,20.06,19.52,19.63,19.7736,889385.29,-2.0948,0.4583,1.0340,-0.42
9,19.63,19.78,19.86,19.17,19.54,19.4986,714452.30,-0.4585,0.3682,0.8307,-0.09


target 0.0471204188481673
********************************************************
(Timestamp('2021-01-04 09:30:00'), 2)
data


,0,1,2,3,4,5,6,7,8,9,10
0,30.91,30.91,31.05,30.44,30.87,30.7860,453121.11,-0.1294,0.4664,0.7002,-0.04
1,30.87,30.78,31.30,30.63,31.04,30.9866,665386.92,0.5507,0.6850,1.0282,0.17
2,31.04,31.00,31.40,30.60,30.85,30.8916,493782.72,-0.6121,0.5083,0.7630,-0.19
3,30.85,31.00,31.29,30.60,30.80,30.9638,525684.71,-0.1621,0.5411,0.8123,-0.05
4,30.80,30.78,31.07,30.12,30.94,30.6191,584039.77,0.4545,0.6012,0.9025,0.14
5,30.94,31.10,31.90,31.09,31.61,31.4056,667012.80,2.1655,0.6866,1.0307,0.67
6,31.61,31.70,32.73,30.70,30.70,31.6939,1353477.49,-2.8788,1.3933,2.0914,-0.91
7,30.70,30.81,30.96,30.14,30.75,30.5316,849853.42,0.1629,0.8748,1.3132,0.05
8,30.75,30.89,31.96,30.71,31.13,31.3922,954561.66,1.2358,0.9826,1.4750,0.38
9,31.13,31.37,31.43,30.55,30.90,30.9365,533327.53,-0.7388,0.5490,0.8241,-0.23


target 0.0387323943661974
********************************************************
(Timestamp('2021-01-04 09:30:00'), 4)
data


...

,0,1,2,3,4,5,6,7,8,9,10
0,6.07,6.05,6.09,6.02,6.09,6.0570,36681.00,0.3295,0.6406,1.3997,0.02
1,6.09,6.10,6.20,6.09,6.16,6.1532,71913.40,1.1494,1.2558,2.7442,0.07
2,6.16,6.16,6.17,6.10,6.15,6.1309,37112.00,-0.1623,0.6481,1.4162,-0.01
3,6.15,6.15,6.32,6.13,6.19,6.2341,100570.01,0.6504,1.7562,3.8377,0.04
4,6.19,6.19,6.19,6.08,6.10,6.1171,49647.53,-1.4540,0.8670,1.8945,-0.09
5,6.10,6.09,6.14,6.05,6.12,6.0899,36138.69,0.3279,0.6311,1.3790,0.02
6,6.12,6.10,6.17,6.08,6.13,6.1327,46002.00,0.1634,0.8033,1.7554,0.01
7,6.13,6.11,6.15,6.05,6.14,6.1074,40143.00,0.1631,0.7010,1.5318,0.01
8,6.14,6.18,6.18,6.11,6.13,6.1245,36030.55,-0.1629,0.6292,1.3749,-0.01
9,6.13,6.10,6.45,6.06,6.29,6.2946,136510.33,2.6101,2.3838,5.2092,0.16


target -0.0958083832335329
********************************************************
(Timestamp('2021-01-04 09:30:00'), 423)
data


,0,1,2,3,4,5,6,7,8,9,10
0,42.19,41.90,44.28,41.90,43.82,43.6995,80056.51,3.8635,1.2244,1.8007,1.63
1,43.82,43.87,44.30,43.41,43.70,43.7980,49109.75,-0.2738,0.7511,1.1046,-0.12
2,43.70,43.70,44.69,43.35,43.60,43.8701,40566.05,-0.2288,0.6204,0.9125,-0.10
3,43.60,43.80,44.08,42.39,42.55,43.0471,48512.04,-2.4083,0.7419,1.0912,-1.05
4,42.55,42.66,43.17,42.35,42.69,42.6289,25893.50,0.3290,0.3960,0.5824,0.14
5,42.69,42.65,42.70,41.10,41.83,41.7424,47102.04,-2.0145,0.7204,1.0595,-0.86
6,41.83,41.98,42.28,41.37,41.71,41.8589,44825.51,-0.2869,0.6855,1.0083,-0.12
7,41.71,41.98,42.85,41.71,41.95,42.1610,55121.83,0.5754,0.8430,1.2399,0.24
8,41.95,42.70,42.96,41.80,42.88,42.4204,46857.95,2.2169,0.7166,1.0540,0.93
9,42.88,42.50,44.42,42.45,43.86,43.8527,80068.74,2.2854,1.2245,1.8010,0.98


target -0.0178340656500387
********************************************************
(Timestamp('2021-01-04 09:30:00'), 425)
data


KeyboardInterrupt: 

In [ ]:
data = data.set_index(["timestamp","ticker"])
x = np.array(data)
x = x.reshape(x.shape[0],-1,3)
for i in range(len(data.index)):
    print(data.index[i])
    display(pd.DataFrame(x[i]))
    print("********************************************************")

In [ ]:
path_list = ['Cov','Corr','Std','Zscore','Return','Decay']
def reshape(data_path,feat_num):
    data = pd.read_parquet(data_path)
    data = np.array(data.set_index(["timestamp","ticker"]))
    data = data.reshape(data.shape[0],feat_num,-1)
    return torch.from_numpy(data)
feat_cat = []
for i in path_list:
    if i in ["Cov","Corr"]:
        feat_cat.append(reshape(path + i +"/" + data_time,conv_num))
    else:
        feat_cat.append(reshape(path + i +"/" + data_time,feat_num))
feat_cat = torch.cat(feat_cat, axis=1)
feat_cat.shape